# 3D_Houses_Project

### Brief presentation of the project :

The objectif was to be able to build a program that is able to take in input a single home address and to return its 3D Model.

### Information concerning the data :

The client had instructed us to use one specific set of data that you can find back here:
* [DSM](http://www.geopunt.be/download?container=dhm-vlaanderen-ii-dsm-raster-1m&title=Digitaal%20Hoogtemodel%20Vlaanderen%20II,%20DSM,%20raster,%201m)

* [DTM](http://www.geopunt.be/download?container=dhm-vlaanderen-ii-dtm-raster-1m&title=Digitaal%20Hoogtemodel%20Vlaanderen%20II,%20DTM,%20raster,%201m)

The data (.TIFF files) is made out of Lidar cloud points, and is mapping the region of Flanders Belgium. 
Therefore 

### Work-flow

* 1 / Import Libraries
* 2 / Have a first adress input
* 3 / Get the coordinates & the polygon/shape of the building
* 4 / Find the appropriate files with the maps and data concerning the adress input
* 5 / Open the appropiate files and crop it to the shape of the building
* 6 / calculate a CHM for this specific building
* 7 / Create and display the house in 3D


### 1 / Import Libraries

In [1]:
#First lets import the libraries we will need to make the program work

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio.plot import show
from rasterio.plot import show_hist
from shapely.geometry import Polygon, mapping
from rasterio.mask import mask
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import geopandas as gpd
import fiona

from matplotlib.colors import ListedColormap
import matplotlib.colors as colors

import requests
import json

import plotly.graph_objects as go



## Address Input

In [2]:
# Here I thought about it as a user interface that the Tourism Office of Flanders could use for their visitor and/or inhabitants
# The user enter an address in Flanders as first input

In [3]:
print('Welcome in Flanders! \nTry out our new 3D House modeling program and visualize any building in Flanders!\nJust enter your address and press start!!!\n\nPlease enter the folowing informations and get your 3D house model :')


Welcome in Flanders! 
Try out our new 3D House modeling program and visualize any building in Flanders!
Just enter your address and press start!!!

Please enter the folowing informations and get your 3D house model :


In [ ]:
city_postcode=input('Postcode : ')

In [ ]:
streetname=input('Streetname (in dutch please) : ')

In [ ]:
house_number= input('The house number :')

## House coordinates & Polygon

To get the House coordinates & Polygon i used an API from the Flemish government that you can find more information about following this link : https://docs.basisregisters.vlaanderen.be/

In [ ]:
# Using the address input data, to get first the coordinates and then the polygon of the house

r = requests.get("https://api.basisregisters.vlaanderen.be/v1/adresmatch", params={"postcode":city_postcode, "straatnaam":streetname, "huisnummer":house_number})
json_test = json.loads(r.content)
json_test

In [ ]:
objectId = json_test['adresMatches'][0]['adresseerbareObjecten'][0]['objectId']

In [ ]:
objectId

In [ ]:
# Coordinates of the house (x,y)
house_coordinates = json_test['adresMatches'][0]['adresPositie']['point']['coordinates']
house_coordinates

In [ ]:
# Coordinates of the X-axis
x_house_coordinates = house_coordinates[0]

In [ ]:
# Coordinates of the Y-axis
y_house_coordinates = house_coordinates[1]

In [ ]:
r = requests.get(f"https://api.basisregisters.vlaanderen.be/v1/gebouweenheden/{objectId}")
json_test_gebouwheden = json.loads(r.content)
json_test_gebouwheden

In [ ]:
object_Id_gebouw = json_test_gebouwheden['gebouw']['objectId']
object_Id_gebouw 

In [ ]:
r = requests.get(f"https://api.basisregisters.vlaanderen.be/v1/gebouwen/{object_Id_gebouw }")
json_test_gebouw = json.loads(r.content)
json_test_gebouw

In [ ]:
#Getting the polygon shapes as a type Dict
shapes = json_test_gebouw['geometriePolygoon'].get('polygon')

## Choosing the adequat raster/.TIFF file

In [ ]:
# house coordinates

house_coordinates
x_house_coordinates
y_house_coordinates

In [ ]:
#There is 43 different raster
#First we need to get the raster boundaries 
#To do so we will use the shapefile (.shp) of each different raster

df =pd.DataFrame(columns=['x1','y1','x2','y2','zone'])
for i in range(1,2):
    if i<10:
        file ='/mnt/c/users/medimonster/documents/github/ant-theano-2-27/projects/data-3d-house/dsm/dsm_0'+str(i)+'/DHMVII_vdc_k0'+str(i)+'.shp'
        #We open .shp files to get the raster boundaries
        c= fiona.open(file)
        x1=c.bounds[0]
        y1=c.bounds[1]
        x2=c.bounds[2]
        y2=c.bounds[3]
        bound=pd.DataFrame([[x1,y1,x2,y2,i]], columns=['x1','y1','x2','y2','zone'])
        #create a dataframe to store these boundaries
        df=pd.concat([df,bound])
        
    else:
        file ='/mnt/c/users/medimonster/documents/github/ant-theano-2-27/projects/data-3d-house/dsm/dsm_'+str(i)+'/DHMVII_vdc_k'+str(i)+'.shp'
        # We open .shp files to get the raster boundaries
        c= fiona.open(file)
        x1=c.bounds[0]
        y1=c.bounds[1]
        x2=c.bounds[2]
        y2=c.bounds[3]
        bound=pd.DataFrame([[x1,y1,x2,y2,i]], columns=['x1','y1','x2','y2','zone'])
        #create a dataframe to store these boundaries
        df=pd.concat([df,bound])


# reset to sequential index
df=df.reset_index(drop=True)


In [ ]:
#choosing the raster using rasters boundaries and house coordinates
zone_bounds=df[(df['x1'] < x_house_coordinates) & (df['x2'] > x_house_coordinates) & (df['y1'] < y_house_coordinates) & (df['y2'] > y_house_coordinates)]


In [ ]:
zone_number=zone_bounds.zone.iloc[0]

## Open TIFF files & Mask 

In [ ]:
# Opening of the adequate files 
# Masking using the shapes/Polygon of the house we got via the API
#Masking allow us to have only the specific information we need for one house (and not the ALL map)

#We will repeat this aswell for the DSM, aswell for the DTM
#The objective is to be able to calculate the CHM (Canopy Height Model) (CHM = DSM - DTM)

In [ ]:
#Here the condition if <10 is only to deal with the difference on how the filenames are written

if zone_number < 10:
    with rio.open('/mnt/c/users/medimonster/documents/github/ant-theano-2-27/projects/data-3d-house/dsm/dsm_0'+str(zone_number)+'/geotiff/DHMVIIDSMRAS1m_k0'+str(zone_number)+'.tif') as src:
        out_image_DSM, out_transform = rio.mask.mask(src, [shapes], crop=True)
        #out_meta =src.meta
        DSM=out_image_DSM[0]
    
    with rio.open('/mnt/c/users/medimonster/documents/github/ant-theano-2-27/projects/data-3d-house/dtm/dtm_0'+str(zone_number)+'/geotiff/DHMVIIDTMRAS1m_k0'+str(zone_number)+'.tif') as src:
        out_image_DTM, out_transform = rio.mask.mask(src, [shapes], crop=True)
        #out_meta =src.meta
        DTM=out_image_DTM[0]
    
    
else:
    with rio.open('/mnt/c/users/medimonster/documents/github/ant-theano-2-27/projects/data-3d-house/dsm/dsm_'+str(zone_number)+'/geotiff/DHMVIIDSMRAS1m_k'+str(zone_number)+'.tif') as src:
        out_image_DSM, out_transform = rio.mask.mask(src, [shapes], crop=True)
        #out_meta =src.meta
        DSM=out_image_DSM[0]
    
    with rio.open('/mnt/c/users/medimonster/documents/github/ant-theano-2-27/projects/data-3d-house/dtm/dtm_'+str(zone_number)+'/geotiff/DHMVIIDTMRAS1m_k'+str(zone_number)+'.tif') as src:
        out_image_DTM, out_transform = rio.mask.mask(src, [shapes], crop=True)
        #out_meta =src.meta
        DTM=out_image_DTM[0]

## Creating the CHM

In [2]:
# Calculating the CHM
out_image_CHM= DSM - DTM

NameError: name 'DSM' is not defined

## 2D Model

In [ ]:
#The house plot in 2D, we can see the difference of height with the difference of colours in the image.

ep.plot_bands(out_image_CHM,
             cmap='terrain',
              title='masked chm\nk01')
plt.show()

## 3D House Model (with Plotly)

In [ ]:
# Our final output the model of the house in 3 dimensions!

fig = go.Figure(data=go.Surface(z=out_image_CHM))
fig.show()

## Reverse mirror image

In [ ]:
# After looking on GoogleMap i could notice that the 3D models of the house were appearing as a mirror image of the real house
# Therefore i add the second line to adjust it -- fig.update_scenes(yaxis_autorange="reversed") --

In [ ]:
fig = go.Figure(data=go.Surface(z=out_image_CHM))
fig.update_scenes(yaxis_autorange="reversed")
fig.show()

## Further development and possibilities for this project

In [ ]:
# Add the nice-to-have features 
#Green areas
#Area of the house
#Pool

#Use OOP to really make it in one click and have a nice interface 
#"hidding" the code and just using the pre programed function to launch the program
